In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score
import json
import numba
from functools import partial
from pandas.io.json import json_normalize

from dsb2019.models.tracking import track_experiment, track_submission_info
from dsb2019.data.validation import InstallationFold, cross_validate, quad_kappa
from dsb2019.visualization import session_browser
from dsb2019.data import DATA_DIR
from dsb2019.models import MODELS_DIR
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import hyperopt
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK
tqdm.pandas()
pd.options.display.max_rows=999

In [3]:
train = pd.read_csv(DATA_DIR / 'raw/train.csv')
test = pd.read_csv(DATA_DIR / 'raw/test.csv')
train_labels = pd.read_csv(DATA_DIR / 'raw/train_labels.csv')
submission = pd.read_csv(DATA_DIR / 'raw/sample_submission.csv')

In [4]:
games = ['Scrub-A-Dub', 'All Star Sorting', 'Mushroom Sorter (Assessment)',
       'Air Show', 'Crystals Rule', 'Bird Measurer (Assessment)',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Cauldron Filler (Assessment)', 'Pan Balance', 'Happy Camel',
       'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)',
       'Leaf Leader']


def unwrap_event_data(df):
    unwrapped = json_normalize(df.event_data.apply(json.loads))
    return pd.concat([unwrapped.reset_index(),df.reset_index()],axis=1)


def process_installations(train_labels, train, process_log):
    result = []
    train["timestamp"] = pd.to_datetime(train.timestamp)
    train = train.drop(["event_count"], axis=1)
    train=train.sort_values("timestamp")
    installations = train.groupby("installation_id")
    for i, game_session, title, installation_id, accuracy_group in tqdm(train_labels[["game_session", "title", "installation_id", "accuracy_group"]].itertuples(), 
                                                              total=len(train_labels)):
        player_log = installations.get_group(installation_id).reset_index()
        log_length = player_log[(player_log.game_session==game_session) & (player_log.title==title)].index[0]
        player_log = player_log.iloc[:(log_length + 1)]
        player_log["accuracy_group"] = accuracy_group
        player_log["target_game_session"] = game_session
        features = process_log(player_log)
        features["installation_id"] = installation_id
        features["accuracy_group"] = accuracy_group
        result.append(features)
    return result


def calculate_ratios(df):
    n_correct=df.correct_move.sum()
    n_incorrect=df.wrong_move.sum()
    ratio=n_correct/(n_correct+n_incorrect)
    return n_correct, n_incorrect, ratio


def make_move_stats(df, title,n_lags=2):
    result = []
    result.extend(zip(["n_correct " + title, "n_incorrect " + title, "global_ratio " + title], calculate_ratios(df)))
    if n_lags:
        last_sessions = df.game_session.unique()[-n_lags:]
        for i in range(n_lags):
            if i < len(last_sessions): 
                result.extend(zip(["n_correct {} {}".format(title,i), "n_incorrect {} {}".format(title,i),"ratio {} {}".format(title,i)], 
                                  calculate_ratios(df[df.game_session==last_sessions[i]])))
            else:
                result.extend(zip(["n_correct {} {}".format(title,i), "n_incorrect {} {}".format(title,i),
                                   "ratio {} {}".format(title,i)], [None, None, None]))
    
    return {k: v for k,v in result}


def shrink_session(group):
    group = populate_correct_moves(group)
    correct_moves = group[group.correct_move]
    correct_timestamps = correct_moves.timestamp
    correct_turns = correct_moves.event_count
    time_between_correct_moves = (correct_timestamps - correct_timestamps.shift(1)).dropna() / np.timedelta64(1, "m")
    turns_between_correct_moves = (correct_turns - correct_turns.shift(1)).dropna()
    result = {
        "mean_time_between_correct_moves": time_between_correct_moves.mean(),
        "mean_turns_between_correct_moves": turns_between_correct_moves.mean()
    }
    if len(correct_timestamps):
        result["time_before_first_correct_move"] = (correct_timestamps.iloc[0] - group.timestamp.iloc[0]) / np.timedelta64(1, "m")
        result["turns_before_first_correct_move"] = correct_turns.iloc[0]
    else:
        result["time_before_first_correct_move"] = None
        result["turns_before_first_correct_move"] = None
    result["start_time"] = group.timestamp.min()
    result["end_time"] = group.timestamp.max()
    result["duration"] = result["end_time"] - result["start_time"]
    result["correct_move"] = group.correct_move.sum()
    result["wrong_move"] = group.wrong_move.sum()
    result["title"] = group.title.iloc[0]
    result["installation_id"] = group.installation_id.iloc[0]
    result["game_session"] = group.game_session.iloc[0]
    return result


def populate_correct_moves(history: pd.DataFrame) -> pd.DataFrame:
    if "correct" in history.columns:
        history["correct_move"] = history.correct == True
        history["wrong_move"] = history.correct == False
    else:
        history["correct_move"]=False
        history["wrong_move"]=False
    return history

    
def make_base_time_features(assessment, history):
    start_end_times = history
    duration_minutes = start_end_times.duration / np.timedelta64(1, "m")
    result = {
        "minutes_played": round(duration_minutes.sum(), 0),
        "mean_session_time_minutes": duration_minutes.mean(), 
    }
    last_event_time = start_end_times.end_time.max()
    first_event_time = start_end_times.start_time.min()
    if assessment is not None:
        result["minutes_since_last_game"] = round((assessment.timestamp - last_event_time) / np.timedelta64(1, "m"), 0)
    
    days_active = round((last_event_time - first_event_time) / np.timedelta64(1, "D"), 0) + 1
    result["games_per_day"] = history.game_session.nunique() / days_active
    
    minutes_between_games = ((start_end_times.start_time - start_end_times.start_time.shift(1)).dropna() / np.timedelta64(1, "m")).round(0)
    result["mean_minutes_between_games"] = minutes_between_games.mean()
    return result
    
    
def apply_on_sessions(history, n_lags, get_features):
    result = {}
    empty = history.head(0)
    if n_lags:
        last_sessions = history.game_session.unique()[-n_lags:]
        for i in range(n_lags):
            if i < len(last_sessions):
                lag_features = get_features(history[history.game_session==last_sessions[i]])
            else:
                lag_features = get_features(empty)
            features = {}
            for k, v in lag_features.items():
                features["%s %d" % (k, i)] = v
            result.update(features)
    return result


def make_session_time_features(df):
    return make_base_time_features(None, df)


def make_game_time_features(df, title, n_lags=2):
    result = apply_on_sessions(df, n_lags, make_session_time_features)
    res = {}
    for k, v in result.items():
        res[title + " " + k] = v
    
    return res


def make_calendar_features(assessment, history):
    ts = assessment.timestamp
    year = ts.year
    month = ts.month
    dayofweek = ts.dayofweek
    time = ts.time()
    return {
        "year": year,
        "month": month,
        "dayofweek": dayofweek,
        "hour": time.hour,
    }


def make_base_features(assessment, history):
    return  {
        "title": games.index(assessment.title)
    }

base_stats = [populate_correct_moves]
base_features = [make_base_features, make_calendar_features, make_base_time_features]
game_features = [make_move_stats, make_game_time_features]

def process_log(df):
    assessment = df.iloc[-1]
    history = df.iloc[:-1]
    history = history[history.type.isin(["Game", "Assessment"])].copy()
    
    if len(history):
        history = unwrap_event_data(history)
    else:
        return {}
    history = json_normalize(history.groupby("game_session").apply(shrink_session))
    for f in base_stats:
        history = f(history)
    result = {}
    for f in base_features:
        result.update(f(assessment, history))
    for game in games:
        stats=history[history.title==game]
        for f in game_features:
            result.update(f(stats, game))
    return result

In [ ]:
train_features= process_installations(train_labels, train, process_log)

  1%|          | 111/17690 [00:29<1:11:35,  4.09it/s]

In [ ]:
train_features.to_csv(DATA_DIR / "interim/train_features_time_features.csv")

In [10]:
train_features=train_features.fillna(-1)

In [12]:
train_features.columns

Index(['title', 'n_correct Scrub-A-Dub', 'n_incorrect Scrub-A-Dub',
       'global_ratio Scrub-A-Dub', 'n_correct Scrub-A-Dub 0',
       'n_incorrect Scrub-A-Dub 0', 'ratio Scrub-A-Dub 0',
       'n_correct Scrub-A-Dub 1', 'n_incorrect Scrub-A-Dub 1',
       'ratio Scrub-A-Dub 1',
       ...
       'n_incorrect Leaf Leader', 'global_ratio Leaf Leader',
       'n_correct Leaf Leader 0', 'n_incorrect Leaf Leader 0',
       'ratio Leaf Leader 0', 'n_correct Leaf Leader 1',
       'n_incorrect Leaf Leader 1', 'ratio Leaf Leader 1', 'installation_id',
       'accuracy_group'],
      dtype='object', length=147)

In [96]:
def lgb_quad_kappa(preds, true):
    true = true.get_label()
    preds = preds.reshape((4, -1)).argmax(axis=0)
    return "quad_kappa", quad_kappa(true, preds), True
    
    
def train_baseline(x_train,y_train, params=None):
    x_train_all, x_val_all,y_train_all,y_val_all = train_test_split(
        x_train,y_train,
        test_size=0.15,
        random_state=2019,
    )
    train_set = lgb.Dataset(x_train_all, y_train_all)
    val_set = lgb.Dataset(x_val_all, y_val_all)

    return lgb.train(params, train_set, num_boost_round=10000, early_stopping_rounds=2000, valid_sets=[train_set, val_set], verbose_eval=100,
                    feval=lgb_quad_kappa)


def make_features_wrapper(train, test):
    def make_features(df):
        return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values
    
    return make_features(train), make_features(test) 


def make_predictions(model,x_test_all,y_test):
    pred=model.predict(x_test_all).argmax(axis=1)
    return pred,y_test

In [16]:
predictions = cross_validate(train_features, train_features.accuracy_group, make_features_wrapper, train_baseline, make_predictions)

Training until validation scores don't improve for 300 rounds
[100]	training's multi_logloss: 1.07328	valid_1's multi_logloss: 1.07759
[200]	training's multi_logloss: 1.01715	valid_1's multi_logloss: 1.03323
[300]	training's multi_logloss: 0.984614	valid_1's multi_logloss: 1.01267
[400]	training's multi_logloss: 0.96036	valid_1's multi_logloss: 1.00071
[500]	training's multi_logloss: 0.941326	valid_1's multi_logloss: 0.992596
[600]	training's multi_logloss: 0.925133	valid_1's multi_logloss: 0.987378
[700]	training's multi_logloss: 0.911415	valid_1's multi_logloss: 0.983922
[800]	training's multi_logloss: 0.898807	valid_1's multi_logloss: 0.981824
[900]	training's multi_logloss: 0.887318	valid_1's multi_logloss: 0.980442
[1000]	training's multi_logloss: 0.876624	valid_1's multi_logloss: 0.979759
[1100]	training's multi_logloss: 0.866517	valid_1's multi_logloss: 0.979412
[1200]	training's multi_logloss: 0.856904	valid_1's multi_logloss: 0.979122
[1300]	training's multi_logloss: 0.847715	

In [20]:
np.mean([quad_kappa(true, pred) for pred, true in predictions]), [quad_kappa(true, pred) for pred, true in predictions]

(0.5157051296362365,
 [0.5472283470243167,
  0.4797202169826231,
  0.47854404188486,
  0.5041272227182371,
  0.5055032393246791,
  0.5232401248520073,
  0.5510630168932551,
  0.5215215852330887,
  0.5147618813899562,
  0.531341620059342])

In [144]:
def process_test_installations(test):
    test = test.sort_values("timestamp")
    test=test.groupby("installation_id").progress_apply(process_log).reset_index()
    test.columns = ["installation_id", "features"]
    result = []
    for i, installation_id, feature in test.itertuples():
        result.append(feature)
        feature["installation_id"]=installation_id
    return pd.DataFrame(result).fillna(-1)
test_features=process_test_installations(test)

100%|██████████| 1000/1000 [01:29<00:00, 11.22it/s]


In [145]:
test_features

,title,n_correct Scrub-A-Dub,n_incorrect Scrub-A-Dub,global_ratio Scrub-A-Dub,n_correct Scrub-A-Dub 0,n_incorrect Scrub-A-Dub 0,ratio Scrub-A-Dub 0,n_correct Scrub-A-Dub 1,n_incorrect Scrub-A-Dub 1,ratio Scrub-A-Dub 1,...,n_correct Leaf Leader,n_incorrect Leaf Leader,global_ratio Leaf Leader,n_correct Leaf Leader 0,n_incorrect Leaf Leader 0,ratio Leaf Leader 0,n_correct Leaf Leader 1,n_incorrect Leaf Leader 1,ratio Leaf Leader 1,installation_id
0,10,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,00abaee7
1,13,15,0,1.0,15.0,0.0,1.0,-1.0,-1.0,-1.0,...,11,0,1.000000,11.0,0.0,1.000000,-1.0,-1.0,-1.0,01242218
2,2,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,017c5718
3,2,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,01a44906
4,13,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,01bc6cb6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,fee254cf
996,14,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,16,7,0.695652,16.0,7.0,0.695652,-1.0,-1.0,-1.0,ff57e602
997,13,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,ffc73fb2
998,13,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,ffe00ca8


In [130]:
subtrain_installations=pd.Series(train_features.installation_id.unique()).sample(frac=1., random_state=2019)
subtrain_features=train_features[train_features.installation_id.isin(subtrain_installations.values)].copy()
def check_hyperparams(params):
    print(params)
    if "max_depth" in params:
        params["max_depth"] = int(params["max_depth"])
    if "num_leaves" in params:
        params["num_leaves"] = int(params["num_leaves"])

    train_baseline_with_params = partial(train_baseline, params=params)
    cv=InstallationFold(n_splits=3)
    predictions = cross_validate(subtrain_features, subtrain_features.accuracy_group, make_features_wrapper, train_baseline_with_params, make_predictions,
                                cv=cv)
    return {
        "loss": -np.mean([quad_kappa(true, pred) for pred, true in predictions]),
        "status": STATUS_OK,
        "params": params
    }


def tune(check_params, n_tries=25, n_learning_rate_tries=15, learning_rate=None, n_estimators=10_000):        
    if learning_rate is None:
        learning_rate_space = {
            'learning_rate': hp.loguniform("learning_rate", np.log(0.005), np.log(0.3)),
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'random_state': 2019,
            "n_estimators": n_estimators,

        }
        trials = Trials()
        result = fmin(check_params,
                      learning_rate_space, tpe.suggest, n_learning_rate_tries, trials=trials)
        print(result)
        learning_rate = round(trials.best_trial["result"]["params"]["learning_rate"], 3)

    param_space = {
        'metric': 'multiclass',
        'objective': 'multiclass',
        'num_classes': 4,
        'lambda_l1': hp.uniform("lamba_l1", 1e-10, 1),
        'lambda_l2': hp.uniform("lambda_l2", 1e-10, 1),
        'random_state': 2019,
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": hp.quniform("max_depth", 2, 16, 1),
        "num_leaves": hp.choice("num_leaves", [3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047, 4095]),
        "subsample": hp.quniform("subsample", 0.01, 1, 0.01),
        "feature_fraction": hp.quniform("feature_fraction", 0.01, 1, 0.01),
    }

    trials = Trials()
    fmin(check_params,
         param_space, tpe.suggest, n_tries, trials=trials)
    best_params = trials.best_trial["result"]["params"]
    return best_params

In [ ]:
best_params=tune(check_hyperparams, n_tries=25, n_learning_rate_tries=10)

{'learning_rate': 0.11570871281185176, 'metric': 'multiclass', 'n_estimators': 10000, 'num_classes': 4, 'objective': 'multiclass', 'random_state': 2019}
Training until validation scores don't improve for 2000 rounds
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	training's multi_logloss: 0.661067	training's quad_kappa: 0.707905	valid_1's multi_logloss: 1.01324	valid_1's quad_kappa: 0.506578
[200]	training's multi_logloss: 0.502574	training's quad_kappa: 0.798993	valid_1's multi_logloss: 1.04839	valid_1's quad_kappa: 0.516114
[300]	training's multi_logloss: 0.399462	training's quad_kappa: 0.851942	valid_1's multi_logloss: 1.10065	valid_1's quad_kappa: 0.508758
[400]	training's multi_logloss: 0.3256	training's quad_kappa: 0.88062	valid_1's multi_logloss: 1.15282	valid_1's quad_kappa: 0.498978
[500]	training's multi_logloss: 0.270233	training's quad_kappa: 0.90667	valid_1's multi_logloss: 1.19725	valid_1's quad_kappa: 0.497703
[600]	training's multi_logloss: 0.22533	training's quad_kappa: 0.924775	valid_1's multi_logloss: 1.24463	valid_1's quad_kappa: 0.485013
[700]	training's multi_logloss: 0.189667	training's quad_kappa: 0.938541	valid_1's multi_logloss: 1.29599	valid_1's quad_kappa: 0.478301
[800]	training's multi_logloss: 0.158597	train

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	training's multi_logloss: 0.675689	training's quad_kappa: 0.702704	valid_1's multi_logloss: 1.01196	valid_1's quad_kappa: 0.496554
[200]	training's multi_logloss: 0.520898	training's quad_kappa: 0.789366	valid_1's multi_logloss: 1.04848	valid_1's quad_kappa: 0.508913
[300]	training's multi_logloss: 0.420943	training's quad_kappa: 0.838961	valid_1's multi_logloss: 1.09075	valid_1's quad_kappa: 0.51054
[400]	training's multi_logloss: 0.346808	training's quad_kappa: 0.873906	valid_1's multi_logloss: 1.13595	valid_1's quad_kappa: 0.501068
[500]	training's multi_logloss: 0.28909	training's quad_kappa: 0.897132	valid_1's multi_logloss: 1.18124	valid_1's quad_kappa: 0.506597
[600]	training's multi_logloss: 0.243317	training's quad_kappa: 0.916788	valid_1's multi_logloss: 1.22112	valid_1's quad_kappa: 0.497956
[700]	training's multi_logloss: 0.205752	training's quad_kappa: 0.934839	valid_1's multi_logloss: 1.26479	valid_1's quad_kappa: 0.499423
[800]	training's multi_logloss: 0.175802	tr

...

[100]	training's multi_logloss: 1.13529	training's quad_kappa: 0.0241339	valid_1's multi_logloss: 1.13719	valid_1's quad_kappa: 0.0232643
[200]	training's multi_logloss: 1.07607	training's quad_kappa: 0.349008	valid_1's multi_logloss: 1.09029	valid_1's quad_kappa: 0.330266
[300]	training's multi_logloss: 1.03769	training's quad_kappa: 0.43139	valid_1's multi_logloss: 1.06349	valid_1's quad_kappa: 0.397931
[400]	training's multi_logloss: 1.00642	training's quad_kappa: 0.484159	valid_1's multi_logloss: 1.04316	valid_1's quad_kappa: 0.445232
[500]	training's multi_logloss: 0.982052	training's quad_kappa: 0.507941	valid_1's multi_logloss: 1.02883	valid_1's quad_kappa: 0.450097
[600]	training's multi_logloss: 0.961821	training's quad_kappa: 0.520356	valid_1's multi_logloss: 1.01846	valid_1's quad_kappa: 0.471518
[700]	training's multi_logloss: 0.945369	training's quad_kappa: 0.535463	valid_1's multi_logloss: 1.01118	valid_1's quad_kappa: 0.481435
[800]	training's multi_logloss: 0.931082	tra

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	training's multi_logloss: 1.12819	training's quad_kappa: 0.298741	valid_1's multi_logloss: 1.11923	valid_1's quad_kappa: 0.283521
[200]	training's multi_logloss: 1.08791	training's quad_kappa: 0.34761	valid_1's multi_logloss: 1.08619	valid_1's quad_kappa: 0.319993
[300]	training's multi_logloss: 1.06441	training's quad_kappa: 0.357388	valid_1's multi_logloss: 1.06984	valid_1's quad_kappa: 0.32793
[400]	training's multi_logloss: 1.04494	training's quad_kappa: 0.372913	valid_1's multi_logloss: 1.05489	valid_1's quad_kappa: 0.344477
[500]	training's multi_logloss: 1.02858	training's quad_kappa: 0.410204	valid_1's multi_logloss: 1.04298	valid_1's quad_kappa: 0.383513
[600]	training's multi_logloss: 1.01512	training's quad_kappa: 0.443904	valid_1's multi_logloss: 1.03393	valid_1's quad_kappa: 0.40283
[700]	training's multi_logloss: 1.00377	training's quad_kappa: 0.466031	valid_1's multi_logloss: 1.02701	valid_1's quad_kappa: 0.412874
[800]	training's multi_logloss: 0.993926	training's

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	training's multi_logloss: 1.1881	training's quad_kappa: 0	valid_1's multi_logloss: 1.18126	valid_1's quad_kappa: 0
[200]	training's multi_logloss: 1.16114	training's quad_kappa: 0.00473087	valid_1's multi_logloss: 1.15843	valid_1's quad_kappa: 0.00467145
[300]	training's multi_logloss: 1.13588	training's quad_kappa: 0.0574882	valid_1's multi_logloss: 1.13675	valid_1's quad_kappa: 0.0565198
[400]	training's multi_logloss: 1.11581	training's quad_kappa: 0.158511	valid_1's multi_logloss: 1.12074	valid_1's quad_kappa: 0.165525
[500]	training's multi_logloss: 1.10088	training's quad_kappa: 0.220927	valid_1's multi_logloss: 1.10934	valid_1's quad_kappa: 0.227909
[600]	training's multi_logloss: 1.08705	training's quad_kappa: 0.267114	valid_1's multi_logloss: 1.09926	valid_1's quad_kappa: 0.261057
[700]	training's multi_logloss: 1.07477	training's quad_kappa: 0.305333	valid_1's multi_logloss: 1.09043	valid_1's quad_kappa: 0.300018
[800]	training's multi_logloss: 1.06288	training's quad_k

In [133]:
best_params

{'feature_fraction': 0.58,
 'lambda_l1': 0.45619796864269707,
 'lambda_l2': 0.033257384218246686,
 'learning_rate': 0.007,
 'max_depth': 14,
 'metric': 'multiclass',
 'n_estimators': 10000,
 'num_classes': 4,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': 2019,
 'subsample': 0.9500000000000001}

What was selected on 20% of the data

```{'feature_fraction': 0.27,
 'lambda_l1': 0.9296219935562766,
 'lambda_l2': 0.9285156686215876,
 'learning_rate': 0.222,
 'max_depth': 5,
 'metric': 'multiclass',
 'n_estimators': 10000,
 'num_classes': 4,
 'num_leaves': 15,
 'objective': 'multiclass',
 'random_state': 2019,
 'subsample': 0.73}```

What was selected on 100% of the data

```
{'feature_fraction': 0.58,
 'lambda_l1': 0.45619796864269707,
 'lambda_l2': 0.033257384218246686,
 'learning_rate': 0.007,
 'max_depth': 14,
 'metric': 'multiclass',
 'n_estimators': 10000,
 'num_classes': 4,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': 2019,
 'subsample': 0.9500000000000001}```

In [134]:
best_params={'feature_fraction': 0.58,
 'lambda_l1': 0.45619796864269707,
 'lambda_l2': 0.033257384218246686,
 'learning_rate': 0.007,
 'max_depth': 14,
 'metric': 'multiclass',
 'n_estimators': 10000,
 'num_classes': 4,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': 2019,
 'subsample': 0.9500000000000001}

In [135]:
baseline_model=train_baseline(train_features.drop(["installation_id", "accuracy_group"], axis=1), train_features.accuracy_group.values, 
               params=best_params)

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	training's multi_logloss: 1.08977	training's quad_kappa: 0.264106	valid_1's multi_logloss: 1.12916	valid_1's quad_kappa: 0.240477
[200]	training's multi_logloss: 1.01707	training's quad_kappa: 0.477512	valid_1's multi_logloss: 1.07882	valid_1's quad_kappa: 0.425825
[300]	training's multi_logloss: 0.968452	training's quad_kappa: 0.528077	valid_1's multi_logloss: 1.04972	valid_1's quad_kappa: 0.485027
[400]	training's multi_logloss: 0.93349	training's quad_kappa: 0.551051	valid_1's multi_logloss: 1.03293	valid_1's quad_kappa: 0.496872
[500]	training's multi_logloss: 0.905467	training's quad_kappa: 0.567028	valid_1's multi_logloss: 1.02182	valid_1's quad_kappa: 0.504934
[600]	training's multi_logloss: 0.882504	training's quad_kappa: 0.579941	valid_1's multi_logloss: 1.01512	valid_1's quad_kappa: 0.506612
[700]	training's multi_logloss: 0.862744	training's quad_kappa: 0.590794	valid_1's multi_logloss: 1.01108	valid_1's qu

In [142]:
baseline_model.save_model(str(MODELS_DIR / "game_baseline.lgb"))

In [191]:
mean_score, cv_scores = (0.5157051296362365,
 [0.5472283470243167,
  0.4797202169826231,
  0.47854404188486,
  0.5041272227182371,
  0.5055032393246791,
  0.5232401248520073,
  0.5510630168932551,
  0.5215215852330887,
  0.5147618813899562,
  0.531341620059342])
name = "Game baseline"
notebook_path="notebooks/Game baseline.ipynb"
submission_path="notebooks/submissions/Game baseline submission.ipynb"
track_experiment(name, mean_score, cv_scores, notebook_path)
track_submission_info(name, submission_path, 0.445)

,name,time,mean_score,cv_scores,notebook_path,submission_path,submission_score
0,Game baseline,2019-10-30 01:43:03.755180+04:00,0.515705,"[0.5472283470243167, 0.4797202169826231, 0.478...",notebooks/Game baseline.ipynb,notebooks/submissions/Game baseline submission...,0.445
